# Embeddings con LSTM



In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from collections import Counter
import random
from itertools import chain
from pprint import pprint
import numpy as np
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm

In [2]:
cd ..

c:\Users\Usuario.000\Documents\Facultad\Git\2020-2\APIT-2020-2\ProyectoFinal


In [3]:
from helpers.boletines import get_sentences

In [4]:
cd Embeddings/

c:\Users\Usuario.000\Documents\Facultad\Git\2020-2\APIT-2020-2\ProyectoFinal\Embeddings


## Definición de constantes

In [2]:
# Número de GPUs disponibles. Usar 0 para modo CPU.
ngpu = 1

# Semilla a usar en los generadores de números aleatorios
SEED = 42069
# SEED = random.randint(1, 10000) # En caso de requerir más resultados
random.seed(SEED)
torch.manual_seed(SEED)

print("Random Seed: ", SEED)

Random Seed:  42069


Selecciono el tipo de dispositivo a utilizar (gpu o cpu)

In [3]:
# Decide si queremos correr en gpu o cpu
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")
device

device(type='cuda', index=0)

## Obtener corpus

In [7]:
uri = open('./../mongo_uri.txt', 'r', encoding='utf-8').read()

In [8]:
sentences = get_sentences(uri)

AMLO
1. Asegura AMLO que en campaña todo será amor y paz , que los otros candidatos se ahorren sus provocaciones .
377. “ Y tiene que ver con la alianza de el PRI y de el PAN , no hay que olvidar que los de el PRI , los de el PAN , los de el PRIAN se pusieron de acuerdo para nombrar a los consejeros de el INE , que tampoco son blancas palomas y también se pusieron de acuerdo para nombrar a los magistrados de el Tribunal_Electoral , yo lo denuncié en su momento ” , comentó .
753. “ Antes en las elecciones pasadas no podíamos defender nos frente_a los ataques a la guerra sucia , a las calumnias , porque nos cercaban , nos daban espacios o actuaban de manera tendenciosa en los medios convencionales , pero en esta elección la diferencia la están marcando las redes sociales , ya no pueden ” , expresó .
1129. En otro orden de ideas , informó que si hoy fuesen las elecciones ganaría con 25 puntos de ventaja , es bastante el número de mexicanos que lo apoyan , pero hay quienes crearán de maner

Desenvuelvo los grupos de enunciados en un único arreglo con todos los enunciados de todos los candidatos.

In [9]:
corpus = list(chain(*sentences.values()))

Paso los tokens a minúsculas para reducir el tamaño del vocabulario

In [10]:
corpus = [[w.lower() for w in sent] for sent in corpus]
print(f'El corpus consta de {len(corpus)} enunciados.')

El corpus consta de 6889 enunciados.


Selecciono k oraciones de prueba para el modelo

In [11]:
k = 6889
sentences = random.sample(corpus, k=k)
print('\n'.join([' '.join(sent) for sent in sentences[::k//10]]))

y se comprometió a regresar como presidente electo para traer el plan de desarrollo para uruapan para decir cuánto se invertirá en la región y regresará cada seis meses para evaluar el plan .
previamente , lópez_obrador denunció que están haciendo llamadas telefónicas a_el_por_mayor a ciudadanos para difamar lo y describió que las llamadas provienen de número telefónicos de el país y de el extranjero , es un bombardeo de llamadas .
indicó que la zona franca se mantuvo en toda la línea fronteriza en la época de porfirio_díaz , después en la revolución_mexicana , hasta que llegó carlos_salinas_de_gortari que acabó con el apoyo en la frontera .
lamentó la violencia que vive el país y confió en que a_pesar_de la situación , la gente saldrá a votar , porque tiene mucho interés en participar el 1_de_julio .
a su salida , entrevistado por los medios de comunicación , reiteró su llamado a el voto útil , invitando a las y los ciudadanos a participar libremente en el proyecto de coalición , y ce

In [4]:
# corpus juguete
sentences = open('corpus_juguete.txt', 'r', encoding='utf-8').readlines()
sentences = [s.strip().split(' ') for s in sentences]
sentences

[['El', 'gato', 'come', 'croquetas', '.'],
 ['El', 'perro', 'come', 'croquetas', '.'],
 ['El', 'gato', 'come', 'atún', '.'],
 ['El', 'perro', 'come', 'pollo', '.'],
 ['El', 'pollo', 'juega', '.'],
 ['El', 'perro', 'juega', '.'],
 ['El', 'gato', 'juega', '.'],
 ['El', 'pollo', 'juega', '.'],
 ['El', 'atún', 'come', '.'],
 ['El', 'pollo', 'come', '.'],
 ['El', 'gato', 'juega', 'con', 'el', 'perro', '.'],
 ['El', 'gato', 'juega', 'con', 'el', 'pollo', '.'],
 ['El', 'perro', 'juega', 'con', 'el', 'gato', '.'],
 ['El', 'perro', 'juega', 'con', 'el', 'pollo', '.'],
 ['El', 'pollo', 'juega', 'con', 'el', 'perro', '.'],
 ['El', 'pollo', 'juega', 'con', 'el', 'gato', '.']]

## Indexar Corpus

Cuento los tokens en todas las oraciones e imprimo los 40 más frecuentes.

In [5]:
freq = sum([Counter(sent) for sent in sentences], Counter())

In [6]:
n_tipos = len(freq.keys())
print(f'Número de tipos: {n_tipos}')
print(freq.most_common()[::max(n_tipos//25, 1)])

Número de tipos: 11
[('El', 16), ('.', 16), ('juega', 10), ('pollo', 8), ('gato', 7), ('perro', 7), ('come', 6), ('con', 6), ('el', 6), ('croquetas', 2), ('atún', 2)]


Agrego un diccionario para pasar de palabra a índice numérico

In [7]:
w_to_index = {
    w : ix
    for ix, (w, freq) in enumerate(freq.most_common())
    if freq > 1 # No toma en cuenta los hapax
}

In [8]:
vocab_size = len(w_to_index)
vocab_size

11

In [9]:
BOS = '<BOS>'
EOS = '<EOS>'
UNK = '<UNK>'

ixBOS = vocab_size
ixEOS = vocab_size + 1
ixUNK = vocab_size + 2

In [10]:
w_to_index[BOS] = ixBOS
w_to_index[EOS] = ixEOS
w_to_index[UNK] = ixUNK

Calculo el nuevo tamaño del vocabulario después de agregar 3 tokens

In [11]:
vocab_size = len(w_to_index)
vocab_size

14

Creo el diccionario inverso, para convertir de índices a palabras

In [12]:
index_to_w = [ w for w, ix in w_to_index.items() ]

Indexo todo el corpus

In [13]:
def w_to_index_unk(w):
    """
    Le asigna el token UNK a palabras que no aparezcan en el corpus
    """
    try:
        return w_to_index[w] 
    except KeyError:
        return ixUNK

In [14]:
sentences_ix = [
    # Le agrego el inicio y fin de caracter a los enunciados
    [ixBOS] + [ w_to_index_unk(w) for w in sent ] + [ixEOS] 
    for sent in sentences
]

X = [ np.asarray(sent[:-1]) for sent in sentences_ix ]
Y = [ np.asarray(sent[1:]) for sent in sentences_ix ] 

print('Ejemplos X')
print(X[:3])
print('Ejemplos Y')
print(Y[:3])

Ejemplos X
[array([11,  0,  4,  6,  9,  1]), array([11,  0,  5,  6,  9,  1]), array([11,  0,  4,  6, 10,  1])]
Ejemplos Y
[array([ 0,  4,  6,  9,  1, 12]), array([ 0,  5,  6,  9,  1, 12]), array([ 0,  4,  6, 10,  1, 12])]


In [15]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.20)

In [16]:
print(len(X_train))
print(len(X_test))

12
4


Mando los vectores de entrada y salida a tensores en gpu

In [17]:
def to_pytorch_tensor(list_of_lists):
    return [
        torch.from_numpy(l).long().to(device)
        for l in list_of_lists
    ]

In [18]:
X_train = to_pytorch_tensor(X_train)
Y_train = to_pytorch_tensor(Y_train)

X_test = to_pytorch_tensor(X_test)
Y_test = to_pytorch_tensor(Y_test)

## Modelo

### 1. Capa de embedding

### 2. Capa oculta

### 3. Capa de salida

Defino las variables para la red neuronal

In [244]:
# Dimensión de entrada (one-hot), tamaño del vocabulario
D_in = vocab_size

# Dimensión de la capa de embedding
D_emb = 2 # 32

# Dimensión de la capa lstm
D_lstm = 4 # 16

# Dimensión de la capa de salida
D_out = D_in

# Épocas de entrenamiento
num_epochs = 300

# Betas para Adam
beta1 = 0.0001
beta2 = 0.99

# Learning rate
lr =  0.1

In [245]:
class Model(nn.Module):
    def __init__(self, ngpu, D_in, D_emb, D_lstm, D_out):
        super(Model, self).__init__()
        self.ngpu = ngpu
        self.embedding = nn.Embedding(num_embeddings=D_in, embedding_dim=D_emb)#, padding_idx=0)
        self.lstm = nn.LSTM(input_size=D_emb, hidden_size=D_lstm) #, bias=True)#, batch_first=True)
        # self.out_layer = nn.Sequential(
        #     nn.Embedding(num_embeddings=D_in, embedding_dim=D_emb, padding_idx=0),
        #     nn.LSTM(input_size=D_emb, hidden_size=D_lstm, bias=True, batch_first=True),
        #     nn.Linear(in_features=D_lstm, out_features=D_out, bias=True),
        #     nn.Softmax(dim=D_out)
        # )
        self.linear = nn.Linear(in_features=D_lstm, out_features=D_out) #, bias=True)
        # self.out_layer = nn.Softmax(dim=1)
        # self.out_layer = nn.Sequential(
        #     nn.Linear(in_features=D_lstm, out_features=D_out, bias=True),
        #     nn.Softmax(dim=1)
        # )

    def forward(self, sentence):
        T = len(sentence)
        # print(sentence)
        # print(T)

        embeddings = self.embedding(sentence).view(T, 1, -1)
        # print('Embeddings')
        # print(embeddings)

        lstm_out, (ht, ct) = self.lstm(embeddings)
        lstm_out = lstm_out.view(T, -1)
        # print('LSTM_out')
        # print(lstm_out)

        preact_out = self.linear(lstm_out).view(T, -1)
        # print('Preact out')
        # print(preact_out)

        return F.log_softmax(preact_out, dim=1)
        # return self.out_layer(preact_out)

In [246]:
model = Model(ngpu, D_in, D_emb, D_lstm, D_out).to(device)
model

Model(
  (embedding): Embedding(14, 2)
  (lstm): LSTM(2, 4)
  (linear): Linear(in_features=4, out_features=14, bias=True)
)

In [247]:
# Inicialización de pesos
# def weights_init(m):
#     classname = m.__class__.__name__
#     if classname.find('Embedding') != -1:
#         # Regularizo los pesos
#         n = m.num_embeddings
#         y = 1.0/np.sqrt(n)
#         m.weight.data.uniform_(-y, y)
#     elif classname.find('Linear') != -1:
#         n = m.in_features
#         y = 1.0/np.sqrt(n)
#         m.weight.data.uniform_(-y, y)
#         m.bias.data.fill_(0)
#     elif classname.find('LSTM') != -1:
#         n = m.input_size
#         y = 1.0/np.sqrt(n)
#         m.weight_ih_l0.data.uniform_(-y, y)
#         m.weight_hh_l0.data.uniform_(-y, y)
#         m.bias_ih_l0.data.fill_(0)
#         m.bias_hh_l0.data.fill_(0)

In [248]:
# model.apply(weights_init)

Entropía cruzada como optmizador y SGD como optimizador

In [249]:
criterion = nn.NLLLoss()
criterion.to(device)
# criterion = nn.NLLLoss()
# optimizer = torch.optim.Adam(model.parameters(), lr=lr, betas=(beta1, beta2))
optimizer = torch.optim.SGD(model.parameters(), lr=lr)

### Entrenamiento

In [250]:
%%time
# for epoch in tqdm(range(num_epochs)):
# for epoch in range(10):
for epoch in range(num_epochs):
    epoch_loss = 0
    for x, y in zip(X_train, Y_train):
    # for x, y in tqdm(zip(X_train, Y_train)):
        if len(x) == 0:
            tqdm.write('Sentencia vacía')
            continue
        # Limpiamos gradientes acumulados
        model.zero_grad()

        # Forward
        # print(x)
        out = model(x)
        # with torch.no_grad():
        #     pred = torch.argmax(out, dim=1)
        #     print('Pred:')
        #     print(pred)
        #     print('y:')
        #     print(y)
        #     print('\n')
        # Step 4. Compute the loss, gradients, and update the parameters by
        #  calling optimizer.step()
        # optimizer.zero_grad()
        loss = criterion(out, y)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        # break
    # tqdm.write(f'Epoch: {epoch:>12} Loss: {loss}')
    # break
    if epoch%20 == 0:
        print(f'Coste después de {epoch+1} épocas: {epoch_loss}')
    # if epoch%10 == 0:
    #     print(list(model.named_parameters()))

Coste después de 1 épocas: 32.912123680114746
Coste después de 21 épocas: 26.16660439968109
Coste después de 41 épocas: 21.43843638896942
Coste después de 61 épocas: 17.623829245567322
Coste después de 81 épocas: 14.907564878463745
Coste después de 101 épocas: 13.018669664859772
Coste después de 121 épocas: 11.261499226093292
Coste después de 141 épocas: 10.043642461299896
Coste después de 161 épocas: 9.256027340888977
Coste después de 181 épocas: 8.716104090213776
Coste después de 201 épocas: 8.32496953010559
Coste después de 221 épocas: 8.0212881565094
Coste después de 241 épocas: 7.772305369377136
Coste después de 261 épocas: 7.565344154834747
Coste después de 281 épocas: 7.389778196811676
Wall time: 16.8 s


In [251]:
# list(model.named_parameters())

In [252]:
def print_ix_sentence(sentence):
    print(' '.join(index_to_w[ix] for ix in sentence.data))

In [253]:
# See what the scores are after training
with torch.no_grad():
    for sentence, y in zip(X_test[:10], Y_test[:10]):
        print('===================================================')
        print('- Probando enunciado:')
        print_ix_sentence(sentence)
        print()

        out = model(sentence)
        prediccion = torch.argmax(out, dim=1)
        print('- Predicción:')
        print_ix_sentence(prediccion)
        print
        print('- Real:')
        print_ix_sentence(y)
        print('\n')

- Probando enunciado:
<BOS> El pollo juega .

- Predicción:
El pollo come . <EOS>
- Real:
El pollo juega . <EOS>


- Probando enunciado:
<BOS> El gato juega .

- Predicción:
El pollo come . <EOS>
- Real:
El gato juega . <EOS>


- Probando enunciado:
<BOS> El perro juega con el pollo .

- Predicción:
El pollo come . el perro . <EOS>
- Real:
El perro juega con el pollo . <EOS>


- Probando enunciado:
<BOS> El perro juega con el gato .

- Predicción:
El pollo come . el perro . <EOS>
- Real:
El perro juega con el gato . <EOS>




In [254]:
# params = list(model.named_parameters())
list(model.named_parameters())

[('embedding.weight',
  Parameter containing:
  tensor([[ 1.5763e+00,  1.8883e+00],
          [-3.1118e-01,  2.9560e+00],
          [ 8.2579e-01,  6.5034e-01],
          [ 9.8333e-01,  2.0115e-03],
          [ 3.1403e+00,  1.0659e+00],
          [ 1.4121e+00, -1.2882e+00],
          [-8.7647e-01, -2.7509e+00],
          [ 1.1388e+00, -1.8062e+00],
          [-1.6106e+00, -1.9376e-01],
          [-9.6354e-02,  1.5616e-01],
          [ 7.8684e-01,  8.0852e-01],
          [-2.4859e+00,  1.8976e+00],
          [ 1.5199e+00, -5.5874e-02],
          [ 7.6869e-01,  9.7801e-01]], device='cuda:0', requires_grad=True)),
 ('lstm.weight_ih_l0',
  Parameter containing:
  tensor([[-0.4469,  0.2968],
          [-0.1957,  0.0548],
          [ 0.9771,  0.0629],
          [-0.0707,  0.2599],
          [-0.0821, -0.0852],
          [ 1.9162, -0.0691],
          [ 0.4111, -0.7245],
          [ 1.2450, -1.0642],
          [-1.5671,  0.3374],
          [ 1.9145,  0.9841],
          [-0.4381,  0.4918],
     

In [255]:
# import matplotlib.pyplot as plt

In [256]:
# plt.scatter(params[0][1].unfold(0, 14, 1)[0], params[0][1].unfold(0, 14, 1)[1])
# plt.show()

In [257]:
# params[0][1].unfold(0, 14, 1)

In [258]:
# params[0][1]